In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import init
import torch.autograd as autograd

import torch.nn as nn
import torch.optim as optim
import torchvision

import copy
import numpy as np
import scipy.stats as ss
import scipy
import h5py
import random
import argparse
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pylab as plt

import warnings
from torch.nn.modules.utils import _single, _pair
import math
import copy
# do crazy end-to-end optimization of backward path with MNIST/CIFAR10 and nn.linear    
# when calling loss.backward both gradinet_weight and grad_weight_feedback are being computed in customized modules
# first control against BP and regular modules
# Linear from here :https://pytorch.org/docs/stable/notes/extending.html
# Conv from here: https://github.com/pytorch/pytorch/blob/master/torch/nn/grad.py    
# autograd : https://pytorch.org/docs/stable/autograd.html#torch.autograd.backward  

class ReLUBFunction(torch.autograd.Function):
    """
    ReLUB (ReLU Back) is a nonlinearity interms of input2 which is used in 
    backward path
    """

    @staticmethod
    def forward(ctx, input, input2):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input, input2)
        input[input2<0] = 0
        return input #input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, input2, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input2 < 0] = 0
        return grad_input, None

class ReLUB(nn.Module):
    def __init__(self):
        super(ReLUB, self).__init__()
    
    def forward(self, input, input2):
        return ReLUBFunction.apply(input, input2)#, input2.clamp(min=0).detach()


import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torchvision.models as models
# from modules import customized_modules_simple as customized_modules
from modules import BiHebb_modules as customized_modules  # used for NeuIPS2020

Linear = customized_modules.Linear
ReLU = nn.ReLU() #customized_modules.ReLU

# class FullyConnectedF(nn.Module):
#     def __init__(self, algorithm):
#         super(FullyConnectedF, self).__init__()
#         input_length = 1024
#         self.fc_0 = Linear(input_length,256,False, algorithm=algorithm)
#         self.fc_1 = Linear(256,256,False, algorithm=algorithm)
#         self.fc_2 = Linear(256,10,False, algorithm=algorithm)
#         self.relu = ReLU
       
#     def forward(self,x):
#         x = self.relu(self.fc_0(x))
#         x = self.relu(self.fc_1(x))
#         x = self.relu(self.fc_2(x))
       
#         return x, x # just to satisfy the requirements

          

# class FullyConnectedB(nn.Module):
#     def __init__(self, hidden_layers, nonlinearfunc, input_length, algorithm):
#         super(FullyConnectedB, self).__init__()
#         input_length = 1024
#         self.fc_0 = Linear(256, input_length,False, algorithm=algorithm)
#         self.fc_1 = Linear(256,256,False, algorithm=algorithm)
#         self.fc_2 = Linear(10,256,False, algorithm=algorithm)
#         self.relu = ReLUB
       
#     def forward(self,x):
        
#         x = self.relu(self.fc_2(x))
#         x = self.relu(self.fc_1(x))
#         x = self.relu(self.fc_0(x))
#         return x, x # just to satisfy the requirements (the first one shoudl be V1 layer)
        


In [3]:
import socket
if socket.gethostname()[0:4] in  ['node','holm','wats']:
    path_prefix = '/rigel/issa/users/Tahereh/Research'
elif socket.gethostname() == 'SYNPAI':
    path_prefix = '/hdd6gig/Documents/Research'
elif socket.gethostname()[0:2] == 'ax':
    path_prefix = '/scratch/issa/users/tt2684/Research'
    plt.switch_backend('agg')
elif socket.gethostname() == 'turing':
    path_prefix = '/home/tahereh/Documents/Research'


imagesetdir = path_prefix+'/Data/'
resultsdir = path_prefix+'/Results/Toy_models/'

class Args:
    dum=None

args = Args()
args.batch_size=256
args.no_cuda = False
args.seed = 0

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

use_cuda = True
# data loader
kwargs = {'num_workers': 0, 'pin_memory': True, 'drop_last':True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1, 0.1),scale=(0.25,2) ),
                        transforms.Resize(32),
                        transforms.ToTensor(),
                       
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=False, transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1, 0.1),scale=(0.25,2) ),
                        transforms.Resize(32),
                        transforms.ToTensor(),
                        
                    ])),
    batch_size=args.batch_size, shuffle=False, **kwargs)

# transforms.Normalize((0.1307,), (0.3081,))
n_layers = 3
n_dataloader = 100000
batch_size = args.batch_size


# class ReLUGrad(nn.Module):
#     def __init__(self):
#         super(ReLUGrad, self).__init__()
#     def forward(self, grad_output, input):
#         grad_input = grad_output.clone()
#         grad_input[input < 0] = 0
#         return grad_input

class Forward(nn.Module):
    def __init__(self, algorithm):
        super(Forward, self).__init__()
        self.fc_0 = Linear(1024, 256, bias=False, algorithm=algorithm)
        self.fc_1 = Linear(256, 256, bias=False, algorithm=algorithm)
        self.fc_2 = Linear(256, 10, bias=False, algorithm=algorithm)
        self.relu = nn.ReLU()

    def forward(self, x):
        xrelu0 = self.fc_0(x)
        x0 = self.relu(xrelu0)
        xrelu1 = self.fc_1(x0)
        x1 = self.relu(xrelu1)
        x2 = self.fc_2(x1)

        return x2 , [x, x0, x1, x2], [xrelu1, xrelu0]

class Backward(nn.Module):
    def __init__(self, algorithm):
        super(Backward, self).__init__()
        self.fc_2 = Linear(10, 256, bias=False, algorithm=algorithm)
        self.fc_1 = Linear(256, 256, bias=False, algorithm=algorithm)
        self.fc_0 = Linear(256, 1024, bias=False, algorithm=algorithm)
        self.grelu = ReLUB()

    def forward(self, x, xrelus):
        x2 = self.fc_2(x)
        x2 = self.grelu(x2, xrelus[0])
        x1 = self.fc_1(x2)
        x1 = self.grelu(x1, xrelus[1])
        x0 = self.fc_0(x1)

        return x0, [x2, x1,  x]

# class Backward(nn.Module):
#     def __init__(self, algorithm):
#         super(Backward, self).__init__()
#         self.fc_2 = Linear(10, 256, bias=False, algorithm=algorithm)
#         self.fc_1 = Linear(256, 256, bias=False, algorithm=algorithm)
#         self.fc_0 = Linear(256, 1024, bias=False, algorithm=algorithm)
#         self.relu = nn.ReLU()

#     def forward(self, x, xrelus):
#         x2 = self.fc_2(x)
#         x2 = self.relu(x2)
#         x1 = self.fc_1(x2)
#         x1 = self.relu(x1)
#         x0 = self.fc_0(x1)

#         return x0, [x2, x1,  x]

def transpose_weights(state_dict):

    state_dict_new = {}
    for k, item in state_dict.items():
        state_dict_new.update({k: item.t()})
    return state_dict_new


def correlation(output, images):
    """Computes the correlation between reconstruction and the original images"""
    x = output.contiguous().view(-1)
    y = images.contiguous().view(-1) 

    vx = x - torch.mean(x)
    vy = y - torch.mean(y)

    pearson_corr = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
    return pearson_corr.item()

In [4]:
algorithm = 'FA' #'BP'
modelF = Forward(algorithm=algorithm).cuda() # main model
modelB = Backward(algorithm=algorithm).cuda() # backward network to compute gradients for modelF

modelC = Forward(algorithm=algorithm).cuda() # Forward Control model to compare to BP
modelC.load_state_dict(modelF.state_dict())
# start symmetric
# modelB.load_state_dict(transpose_weights(modelF.state_dict()) )
modelE = Forward(algorithm=algorithm).cuda()

modelF_untrained = modelF.state_dict()
modelB_untrained = modelB.state_dict()
modelC_untrained = modelC.state_dict()

In [5]:

algorithm = 'FA' #'BP'
modelF = Forward(algorithm=algorithm).cuda() # main model
modelB = Backward(algorithm=algorithm).cuda() # backward network to compute gradients for modelF

modelC = Forward(algorithm=algorithm).cuda() # Forward Control model to compare to BP
modelC.load_state_dict(modelF.state_dict())
# start symmetric
# modelB.load_state_dict(transpose_weights(modelF.state_dict()) )

modelF.load_state_dict(modelF_untrained)
modelB.load_state_dict(modelB_untrained)
modelC.load_state_dict(modelC_untrained)

optimizerC = optim.SGD(modelC.parameters(), lr=1e-1, weight_decay=1e-4)
optimizerF = optim.RMSprop([p for n,p in modelF.named_parameters() if 'feedback' not in n],   lr=1e-3, weight_decay=1e-4)
optimizerB = optim.RMSprop([p for n,p in modelB.named_parameters() if 'feedback' not in n],   lr=1e-3, weight_decay=1e-5)

print(optimizerF.state_dict())

criterionF = nn.CrossEntropyLoss() #
criterionB = nn.MSELoss() #

n_classes = 10
onehot = torch.zeros(train_loader.batch_size, n_classes).cuda()


for epoch in range(10):
    running_loss = 0.0
    running_lossB = 0.0
    running_corrB = 0.0
    acc = 0.0
    for images, targets in train_loader:
        images = images.cuda()
        targets = targets.cuda()

        onehot = torch.zeros(train_loader.batch_size, n_classes).cuda()
        onehot.zero_()
        onehot.scatter_(1, targets.view(train_loader.batch_size,-1), 1)

        images = images.view(images.shape[0], -1)
        out, interm, xrelu = modelF(images)
        loss = criterionF(out, targets)

        
        optimizerF.zero_grad()
        loss.backward()
        optimizerF.step()

        # # SL
        # modelB.load_state_dict(transpose_weights(modelF.state_dict()))
        # out, interm, xrelu = modelF(images)
        # recons, _ = modelB(out.detach() - onehot, xrelu)
        # # recons, _ = modelB(out.detach(), xrelu)
        # optimizerB.zero_grad()
        # lossB = criterionB(recons, images)
        # optimizerB.step()
        # modelF.load_state_dict(transpose_weights(modelB.state_dict()))
    # SL
    modelB.load_state_dict(transpose_weights(modelF.state_dict()))
    for images, targets in train_loader:
        images = images.cuda()
        targets = targets.cuda()

        onehot = torch.zeros(train_loader.batch_size, n_classes).cuda()
        onehot.zero_()
        onehot.scatter_(1, targets.view(train_loader.batch_size,-1), 1)

        images = images.view(images.shape[0], -1)


        out, interm, xrelu = modelF(images)
        # inputB = F.normalize(out.detach() -onehot) 
        inputB = out.detach() -onehot
        inputB = (inputB - inputB.mean().expand_as(inputB))/inputB.std().expand_as(inputB)
        recons, _ = modelB(inputB  , xrelu)
        optimizerB.zero_grad()
        lossB = criterionB(recons, images)
        corrB = correlation(recons, images)
        optimizerB.step()
        modelF.load_state_dict(transpose_weights(modelB.state_dict()))
        running_lossB += lossB.item()
        running_corrB += corrB


        running_loss += loss.item()
        _, predicted = torch.max(out.data, 1)
        acc += (predicted == targets).sum().item()/out.shape[0]

    print(epoch, running_corrB/len(train_loader), acc/len(train_loader) )
    print(torch.norm(modelF.fc_1.weight))


{'state': {}, 'param_groups': [{'lr': 0.001, 'momentum': 0, 'alpha': 0.99, 'eps': 1e-08, 'centered': False, 'weight_decay': 0.0001, 'params': [139746300707904, 139746300707584, 139746300708864]}]}
0 0.3542507990048482 0.5586605235042735
tensor(11.3620, device='cuda:0', grad_fn=<NormBackward0>)
1 0.33724967243834436 0.6285223023504274
tensor(13.3041, device='cuda:0', grad_fn=<NormBackward0>)
2 0.32985374573458975 0.7297509348290598
tensor(14.7630, device='cuda:0', grad_fn=<NormBackward0>)
3 0.3315411569216313 0.7608673878205128
tensor(15.9823, device='cuda:0', grad_fn=<NormBackward0>)
4 0.32991868703283816 0.7784788995726496
tensor(17.0077, device='cuda:0', grad_fn=<NormBackward0>)
5 0.3241402432959304 0.7949719551282052
tensor(17.8578, device='cuda:0', grad_fn=<NormBackward0>)
6 0.331319234182692 0.801582532051282
tensor(18.5772, device='cuda:0', grad_fn=<NormBackward0>)
7 0.3446260267852718 0.7157952724358975
tensor(19.1652, device='cuda:0', grad_fn=<NormBackward0>)
8 0.31799943681456

In [6]:
SL relu out-onehot
0 0.4618158159602402 0.5472923344017094
1 0.3776456688204382 0.6429453792735043
2 0.32424116402100295 0.6848123664529915
3 0.2554798854721917 0.736862313034188
4 0.2367170939588139 0.7501502403846154
5 0.2253640986278526 0.7605168269230769
6 0.1854674427045716 0.7770599626068376
7 0.20750007606469667 0.7900807959401709
8 0.18773908040717116 0.8110142895299145
9 0.1761054252075334 0.7721020299145299

normalized
0 0.42776161139337426 0.5583934294871795
1 0.3757980693864007 0.662910657051282
2 0.30473190571507836 0.7157618856837606
3 0.25142446001116026 0.7382311698717948
4 0.2510691239283635 0.7729200053418803
5 0.22847212507174566 0.7751068376068376
6 0.18480198078940058 0.7708333333333334
7 0.17948726723846206 0.8119157318376068
8 0.18290208426550922 0.8017661591880342
9 0.1943849072369755 0.7871761485042735

SyntaxError: invalid syntax (<ipython-input-6-44993ce23fb0>, line 1)

In [ ]:
SL grelu out-onehot 

0 0.42766048727381945 0.5623831463675214
1 0.37991426006341594 0.634498530982906
2 0.34845178529747534 0.7121895032051282
3 0.3395874370367099 0.7560263087606838
4 0.3336598601860878 0.7611010950854701
5 0.35095278893271065 0.7665097489316239
6 0.34155619895865774 0.7979767628205128
7 0.35036978558597404 0.8031016292735043
8 0.3505960252040472 0.8009982638888888
9 0.35636504822307163 0.8245025373931624

0 0.42715904091158485 0.5602130074786325
1 0.46117274819785714 0.6496728098290598
2 0.41402703892980885 0.6952457264957265
3 0.3697844798493589 0.7417367788461539
4 0.34432112521085984 0.7130408653846154
5 0.33250361477207935 0.7775941506410257
6 0.33055557820022613 0.7970419337606838
7 0.3220561359427933 0.8013321314102564
8 0.33975157867639494 0.7974926549145299
9 0.31651485297414994 0.8157719017094017

In [ ]:
0 0.41600701009106433 0.5765391292735043
1 0.3207030433874864 0.6105769230769231
2 0.3144686948030423 0.7324051816239316
3 0.31064027012922824 0.7684628739316239
4 0.31330710076368773 0.7545072115384616
5 0.3095823609166675 0.7602330395299145
6 0.31068627396200454 0.8077256944444444
7 0.321371576979629 0.8190938835470085
8 0.3265181139238879 0.8200620993589743
9 0.33742810187176764 0.8216980502136753

In [ ]:
0 0.0 0.435947516025641
1 0.0 0.6284722222222222
2 0.0 0.6996026976495726
3 0.0 0.7355602297008547
4 0.0 0.7592481303418803
5 0.0 0.7737379807692307
6 0.0 0.7885950854700855
7 0.0 0.7981270032051282
8 0.0 0.8041366185897436
9 0.0 0.8107138087606838